In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
df1 = pd.DataFrame(columns = ['Subject','DistanceR','DistanceL', 'Gender', 'Label'])
labels = [1, 1, 0, 0] #1: Disabled
gender = [1,0,1,0] #1: Male

In [11]:
#Let us take an example 111GM3
aFileName = 'C:\\Harinee\\Capstone2\\Data\\Recording Data\\111GM3\A1R.txt'
subjectName = '111GM3'
a1rData = pd.read_csv(aFileName, sep="\t", decimal=",")
a1rData

,T,LX,LY,RX,RY
0,0,0.00000,0.00000,0.00000,0.00000
1,20,0.65535,-0.00001,0.65536,-0.65536
2,40,0.65534,-0.00001,0.65536,-0.65536
3,60,0.65534,-0.00001,0.65535,-0.65536
4,80,0.65534,-0.00001,0.65534,0.00000
...,...,...,...,...,...
1495,29900,105.51480,36.04447,106.17018,32.76765
1496,29920,107.48088,27.52472,108.79159,14.41743
1497,29940,108.13612,-16.38473,102.89310,-78.64416
1498,29960,104.20380,-135.00506,114.69085,-155.97572


In [12]:
#Now adding two more features 
#Use distance = sqrt(square(x1-x2) + square(y1-y2))
a1rData['DistanceL'] = np.sqrt((a1rData['LX'].shift(1) - a1rData['LX'])**2 + ((a1rData['LY'].shift(1) - a1rData['LY'])**2))
a1rData['DistanceR'] = np.sqrt((a1rData['RX'].shift(1) - a1rData['RX'])**2 + ((a1rData['RY'].shift(1) - a1rData['RY'])**2))
#Add a fixed value for the Subject id
a1rData['Subject'] = subjectName
a1rData

,T,LX,LY,RX,RY,DistanceL,DistanceR,Subject
0,0,0.00000,0.00000,0.00000,0.00000,NaN,NaN,111GM3
1,20,0.65535,-0.00001,0.65536,-0.65536,0.655350,0.926819,111GM3
2,40,0.65534,-0.00001,0.65536,-0.65536,0.000010,0.000000,111GM3
3,60,0.65534,-0.00001,0.65535,-0.65536,0.000000,0.000010,111GM3
4,80,0.65534,-0.00001,0.65534,0.00000,0.000000,0.655360,111GM3
...,...,...,...,...,...,...,...,...
1495,29900,105.51480,36.04447,106.17018,32.76765,2.362940,4.144892,111GM3
1496,29920,107.48088,27.52472,108.79159,14.41743,8.743661,18.536514,111GM3
1497,29940,108.13612,-16.38473,102.89310,-78.64416,43.914339,93.248334,111GM3
1498,29960,104.20380,-135.00506,114.69085,-155.97572,118.685491,78.226320,111GM3


In [13]:
#Now compute the sum of distances traveled for this subject
aSubjectData = a1rData[['Subject','DistanceR','DistanceL']].groupby('Subject', group_keys=False).sum()
#display the one rwo containing the distance traveled for a specific subject
aSubjectData


,DistanceR,DistanceL
Subject,,
111GM3,13580.30654,13413.296257


In [14]:
#Now create a hardcoded column Subject and derive the label and gender from the subject id
#Subjects with a code ending in 1 or 2 were reading disabled.
#Subjects with a code ending in 3 or 4 were controls. 
#Subjects with a code ending in 1 or 3 were male.
#Subjects with a code ending in 2 or 4 were female.

aSubjectData['Subject'] = subjectName
aSubjectData['Label'] = labels[int(subjectName[-1])-1]
aSubjectData['Gender'] = gender[int(subjectName[-1])-1]
#reset the index to remove the Subject as index
aSubjectData = aSubjectData.reset_index(level=0, drop=True)
aSubjectData

,DistanceR,DistanceL,Subject,Label,Gender
0,13580.30654,13413.296257,111GM3,0,1


In [15]:
#Go back to a1rData and extract few more columns from the eye movement data: Add LType and RType
#S Saccade F Fixation: If the distance travelled is < 1 (review this) then it is Fixation else Saccade
a1rData['LType'] = a1rData['DistanceL'].apply(lambda x: 'F' if x < 1 else 'S')
a1rData['RType'] = a1rData['DistanceR'].apply(lambda x: 'F' if x < 1 else 'S')
a1rData

,T,LX,LY,RX,RY,DistanceL,DistanceR,Subject,LType,RType
0,0,0.00000,0.00000,0.00000,0.00000,NaN,NaN,111GM3,S,S
1,20,0.65535,-0.00001,0.65536,-0.65536,0.655350,0.926819,111GM3,F,F
2,40,0.65534,-0.00001,0.65536,-0.65536,0.000010,0.000000,111GM3,F,F
3,60,0.65534,-0.00001,0.65535,-0.65536,0.000000,0.000010,111GM3,F,F
4,80,0.65534,-0.00001,0.65534,0.00000,0.000000,0.655360,111GM3,F,F
...,...,...,...,...,...,...,...,...,...,...
1495,29900,105.51480,36.04447,106.17018,32.76765,2.362940,4.144892,111GM3,S,S
1496,29920,107.48088,27.52472,108.79159,14.41743,8.743661,18.536514,111GM3,S,S
1497,29940,108.13612,-16.38473,102.89310,-78.64416,43.914339,93.248334,111GM3,S,S
1498,29960,104.20380,-135.00506,114.69085,-155.97572,118.685491,78.226320,111GM3,S,S


In [16]:
#Now see weather the movement is right/left for horizontal or (up/down) for vertical for each of the eye movemtn (left & right)
#-1 left/Down  1 Right/Up 0 No change: Identify the Left and Right horizontal and vertical direction
a1rData['LHorDir'] = (a1rData['LX'].shift(1) - a1rData['LX']).apply(lambda x: 'N' if abs(x) < 0.01 else ('R' if x <= 0 else 'L'))
a1rData['RHorDir'] = (a1rData['RX'].shift(1) - a1rData['RX']).apply(lambda x: 'N' if abs(x) < 0.01 else ('R' if x <= 0 else 'L'))
a1rData['LVerDir'] = (a1rData['LY'].shift(1) - a1rData['LY']).apply(lambda x: 'N' if abs(x) < 0.01 else ('U' if x <= 0 else 'D'))
a1rData['RVerDir'] = (a1rData['RY'].shift(1) - a1rData['RY']).apply(lambda x: 'N' if abs(x) < 0.01 else ('U' if x <= 0 else 'D'))
a1rData

,T,LX,LY,RX,RY,DistanceL,DistanceR,Subject,LType,RType,LHorDir,RHorDir,LVerDir,RVerDir
0,0,0.00000,0.00000,0.00000,0.00000,NaN,NaN,111GM3,S,S,L,L,D,D
1,20,0.65535,-0.00001,0.65536,-0.65536,0.655350,0.926819,111GM3,F,F,R,R,N,D
2,40,0.65534,-0.00001,0.65536,-0.65536,0.000010,0.000000,111GM3,F,F,N,N,N,N
3,60,0.65534,-0.00001,0.65535,-0.65536,0.000000,0.000010,111GM3,F,F,N,N,N,N
4,80,0.65534,-0.00001,0.65534,0.00000,0.000000,0.655360,111GM3,F,F,N,N,N,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,29900,105.51480,36.04447,106.17018,32.76765,2.362940,4.144892,111GM3,S,S,R,R,D,D
1496,29920,107.48088,27.52472,108.79159,14.41743,8.743661,18.536514,111GM3,S,S,R,R,D,D
1497,29940,108.13612,-16.38473,102.89310,-78.64416,43.914339,93.248334,111GM3,S,S,R,L,D,D
1498,29960,104.20380,-135.00506,114.69085,-155.97572,118.685491,78.226320,111GM3,S,S,L,R,D,D


In [17]:
#Create temp columns changes in the LType passed
a1rData['TempSeqChangeL'] = a1rData['LType'].ne(a1rData['LType'].shift(1)).astype(int)
print(a1rData.shape)
a1rData.head(10)

(1500, 15)


,T,LX,LY,RX,RY,DistanceL,DistanceR,Subject,LType,RType,LHorDir,RHorDir,LVerDir,RVerDir,TempSeqChangeL
0,0,0.00000,0.00000,0.00000,0.00000,NaN,NaN,111GM3,S,S,L,L,D,D,1
1,20,0.65535,-0.00001,0.65536,-0.65536,0.655350,0.926819,111GM3,F,F,R,R,N,D,1
2,40,0.65534,-0.00001,0.65536,-0.65536,0.000010,0.000000,111GM3,F,F,N,N,N,N,0
3,60,0.65534,-0.00001,0.65535,-0.65536,0.000000,0.000010,111GM3,F,F,N,N,N,N,0
4,80,0.65534,-0.00001,0.65534,0.00000,0.000000,0.655360,111GM3,F,F,N,N,N,U,0
5,100,0.65533,-0.00001,0.65534,-0.65536,0.000010,0.655360,111GM3,F,F,N,N,N,D,0
6,120,1.31069,-0.00001,0.65534,-0.65536,0.655360,0.000000,111GM3,F,F,R,N,N,N,0
7,140,1.31069,-0.65537,1.31069,-0.65536,0.655360,0.655350,111GM3,F,F,N,R,D,N,0
8,160,1.31068,-0.65537,1.31069,-0.65536,0.000010,0.000000,111GM3,F,F,N,N,N,N,0
9,180,0.65532,-0.00001,1.31069,0.00000,0.926819,0.655360,111GM3,F,F,L,N,U,U,0


In [18]:
#Now locate only the changes - i,e, tempSeqChange = 1
a1rChangesDataL = a1rData.loc[a1rData['TempSeqChangeL'] != 0]
a1rChangesDataL

,T,LX,LY,RX,RY,DistanceL,DistanceR,Subject,LType,RType,LHorDir,RHorDir,LVerDir,RVerDir,TempSeqChangeL
0,0,0.00000,0.00000,0.00000,0.00000,NaN,NaN,111GM3,S,S,L,L,D,D,1
1,20,0.65535,-0.00001,0.65536,-0.65536,0.655350,0.926819,111GM3,F,F,R,R,N,D,1
11,220,-0.65543,3.27683,-1.96614,5.89831,3.276842,6.747406,111GM3,S,S,L,L,U,U,1
13,260,-1.31079,7.86439,-0.65543,7.20903,0.000000,0.926812,111GM3,F,F,N,R,N,D,1
27,540,4.58751,3.93217,3.27680,3.93219,5.404281,1.465438,111GM3,S,S,R,L,U,U,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,29660,122.55412,32.11215,120.58806,32.76752,2.362923,3.276784,111GM3,S,S,L,L,U,U,1
1485,29700,120.58807,32.76753,121.89881,32.11219,0.926826,2.072433,111GM3,F,S,R,R,D,D,1
1486,29720,123.86489,32.11218,123.86491,30.80148,3.341711,2.362945,111GM3,S,S,R,R,D,D,1
1491,29820,102.89331,39.32129,102.23797,39.32130,0.926798,0.655340,111GM3,F,F,L,L,U,N,1


In [19]:
#Now get the difference in T values to get the duration of the Ltypes
a1rChangesDataL['TempDur'] = a1rChangesDataL['T'].shift(-1) - a1rChangesDataL['T']
a1rChangesDataL

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,T,LX,LY,RX,RY,DistanceL,DistanceR,Subject,LType,RType,LHorDir,RHorDir,LVerDir,RVerDir,TempSeqChangeL,TempDur
0,0,0.00000,0.00000,0.00000,0.00000,NaN,NaN,111GM3,S,S,L,L,D,D,1,20.0
1,20,0.65535,-0.00001,0.65536,-0.65536,0.655350,0.926819,111GM3,F,F,R,R,N,D,1,200.0
11,220,-0.65543,3.27683,-1.96614,5.89831,3.276842,6.747406,111GM3,S,S,L,L,U,U,1,40.0
13,260,-1.31079,7.86439,-0.65543,7.20903,0.000000,0.926812,111GM3,F,F,N,R,N,D,1,280.0
27,540,4.58751,3.93217,3.27680,3.93219,5.404281,1.465438,111GM3,S,S,R,L,U,U,1,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,29660,122.55412,32.11215,120.58806,32.76752,2.362923,3.276784,111GM3,S,S,L,L,U,U,1,40.0
1485,29700,120.58807,32.76753,121.89881,32.11219,0.926826,2.072433,111GM3,F,S,R,R,D,D,1,20.0
1486,29720,123.86489,32.11218,123.86491,30.80148,3.341711,2.362945,111GM3,S,S,R,R,D,D,1,100.0
1491,29820,102.89331,39.32129,102.23797,39.32130,0.926798,0.655340,111GM3,F,F,L,L,U,N,1,20.0


In [20]:
#compute the sum of F durations and S durations, grouping by LType
a1rFandSDataL = a1rChangesDataL.groupby(['Subject', 'LType'])['TempDur'].agg('sum')
a1rFandSDataL

Subject  LType
111GM3   F        15320.0
         S        14520.0
Name: TempDur, dtype: float64

In [21]:
a1rFandSDataL = a1rFandSDataL.reset_index(level=0, drop=True)
a1rFandSDataL

LType
F    15320.0
S    14520.0
Name: TempDur, dtype: float64

In [22]:
#Get a dataframe out of the series above
dfL = pd.DataFrame([[a1rFandSDataL['F'], a1rFandSDataL['S']]], columns = ['LTypeFSum', 'LTypeSSum'])
dfL

,LTypeFSum,LTypeSSum
0,15320.0,14520.0


In [23]:
aSubjectData = pd.concat([aSubjectData, dfL], axis=1)
aSubjectData

,DistanceR,DistanceL,Subject,Label,Gender,LTypeFSum,LTypeSSum
0,13580.30654,13413.296257,111GM3,0,1,15320.0,14520.0


In [24]:
#Create temp columns changes in the LType passed
a1rData['TempSeqChangeR'] = a1rData['RType'].ne(a1rData['RType'].shift(1)).astype(int)
print(a1rData.shape)
a1rData.head(10)

(1500, 16)


,T,LX,LY,RX,RY,DistanceL,DistanceR,Subject,LType,RType,LHorDir,RHorDir,LVerDir,RVerDir,TempSeqChangeL,TempSeqChangeR
0,0,0.00000,0.00000,0.00000,0.00000,NaN,NaN,111GM3,S,S,L,L,D,D,1,1
1,20,0.65535,-0.00001,0.65536,-0.65536,0.655350,0.926819,111GM3,F,F,R,R,N,D,1,1
2,40,0.65534,-0.00001,0.65536,-0.65536,0.000010,0.000000,111GM3,F,F,N,N,N,N,0,0
3,60,0.65534,-0.00001,0.65535,-0.65536,0.000000,0.000010,111GM3,F,F,N,N,N,N,0,0
4,80,0.65534,-0.00001,0.65534,0.00000,0.000000,0.655360,111GM3,F,F,N,N,N,U,0,0
5,100,0.65533,-0.00001,0.65534,-0.65536,0.000010,0.655360,111GM3,F,F,N,N,N,D,0,0
6,120,1.31069,-0.00001,0.65534,-0.65536,0.655360,0.000000,111GM3,F,F,R,N,N,N,0,0
7,140,1.31069,-0.65537,1.31069,-0.65536,0.655360,0.655350,111GM3,F,F,N,R,D,N,0,0
8,160,1.31068,-0.65537,1.31069,-0.65536,0.000010,0.000000,111GM3,F,F,N,N,N,N,0,0
9,180,0.65532,-0.00001,1.31069,0.00000,0.926819,0.655360,111GM3,F,F,L,N,U,U,0,0


In [25]:
#Now locate only the changes - i,e, tempSeqChange = 1
a1rChangesDataR = a1rData.loc[a1rData['TempSeqChangeR'] != 0]
a1rChangesDataR

,T,LX,LY,RX,RY,DistanceL,DistanceR,Subject,LType,RType,LHorDir,RHorDir,LVerDir,RVerDir,TempSeqChangeL,TempSeqChangeR
0,0,0.00000,0.00000,0.00000,0.00000,NaN,NaN,111GM3,S,S,L,L,D,D,1,1
1,20,0.65535,-0.00001,0.65536,-0.65536,0.655350,0.926819,111GM3,F,F,R,R,N,D,1,1
11,220,-0.65543,3.27683,-1.96614,5.89831,3.276842,6.747406,111GM3,S,S,L,L,U,U,1,1
13,260,-1.31079,7.86439,-0.65543,7.20903,0.000000,0.926812,111GM3,F,F,N,R,N,D,1,1
16,320,-0.65543,5.89830,-0.65542,5.24294,0.655360,1.465438,111GM3,F,S,N,R,D,D,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,29020,-33.42386,20.97171,-32.76849,21.62708,1.465421,1.853638,111GM3,S,S,R,R,U,U,0,1
1484,29680,119.93270,33.42289,119.93272,32.76753,2.930850,0.655340,111GM3,S,F,L,L,U,N,0,1
1485,29700,120.58807,32.76753,121.89881,32.11219,0.926826,2.072433,111GM3,F,S,R,R,D,D,1,1
1491,29820,102.89331,39.32129,102.23797,39.32130,0.926798,0.655340,111GM3,F,F,L,L,U,N,1,1


In [26]:
#Now get the difference in T values to get the duration of the Ltypes
a1rChangesDataR['TempDur'] = a1rChangesDataR['T'].shift(-1) - a1rChangesDataR['T']
a1rChangesDataR

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,T,LX,LY,RX,RY,DistanceL,DistanceR,Subject,LType,RType,LHorDir,RHorDir,LVerDir,RVerDir,TempSeqChangeL,TempSeqChangeR,TempDur
0,0,0.00000,0.00000,0.00000,0.00000,NaN,NaN,111GM3,S,S,L,L,D,D,1,1,20.0
1,20,0.65535,-0.00001,0.65536,-0.65536,0.655350,0.926819,111GM3,F,F,R,R,N,D,1,1,200.0
11,220,-0.65543,3.27683,-1.96614,5.89831,3.276842,6.747406,111GM3,S,S,L,L,U,U,1,1,40.0
13,260,-1.31079,7.86439,-0.65543,7.20903,0.000000,0.926812,111GM3,F,F,N,R,N,D,1,1,60.0
16,320,-0.65543,5.89830,-0.65542,5.24294,0.655360,1.465438,111GM3,F,S,N,R,D,D,0,1,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,29020,-33.42386,20.97171,-32.76849,21.62708,1.465421,1.853638,111GM3,S,S,R,R,U,U,0,1,660.0
1484,29680,119.93270,33.42289,119.93272,32.76753,2.930850,0.655340,111GM3,S,F,L,L,U,N,0,1,20.0
1485,29700,120.58807,32.76753,121.89881,32.11219,0.926826,2.072433,111GM3,F,S,R,R,D,D,1,1,120.0
1491,29820,102.89331,39.32129,102.23797,39.32130,0.926798,0.655340,111GM3,F,F,L,L,U,N,1,1,40.0


In [27]:
#Now get the difference in T values to get the duration of the RTypes
#compute the sum of F durations and S durations, grouping by RType
a1rFandSDataR = a1rChangesDataR.groupby(['Subject', 'RType'])['TempDur'].agg('sum')
a1rFandSDataR

Subject  RType
111GM3   F        15020.0
         S        14840.0
Name: TempDur, dtype: float64

In [28]:
a1rFandSDataR = a1rFandSDataR.reset_index(level=0, drop=True)
a1rFandSDataR

RType
F    15020.0
S    14840.0
Name: TempDur, dtype: float64

In [29]:
dfR = pd.DataFrame([[a1rFandSDataR['F'], a1rFandSDataR['S']]], columns = ['RTypeFSum', 'RTypeSSum'])
dfR

,RTypeFSum,RTypeSSum
0,15020.0,14840.0


In [30]:
aSubjectData = pd.concat([aSubjectData, dfR], axis=1)
aSubjectData

,DistanceR,DistanceL,Subject,Label,Gender,LTypeFSum,LTypeSSum,RTypeFSum,RTypeSSum
0,13580.30654,13413.296257,111GM3,0,1,15320.0,14520.0,15020.0,14840.0


In [31]:
#compute the mean of F durations and S durations, grouping by RType
a1rFandSDataRmean = a1rChangesDataR.groupby(['Subject', 'RType'])['TempDur'].agg('mean')
a1rFandSDataRmean = a1rFandSDataRmean.reset_index(level=0, drop=True)
dfRmean = pd.DataFrame([[a1rFandSDataRmean['F'], a1rFandSDataRmean['S']]], columns = ['RTypeFMean', 'RTypeSMean'])
aSubjectData = pd.concat([aSubjectData, dfRmean], axis=1)
aSubjectData

,DistanceR,DistanceL,Subject,Label,Gender,LTypeFSum,LTypeSSum,RTypeFSum,RTypeSSum,RTypeFMean,RTypeSMean
0,13580.30654,13413.296257,111GM3,0,1,15320.0,14520.0,15020.0,14840.0,82.076503,81.092896


In [32]:
#compute the mean of F durations and S durations, grouping by LType
a1rFandSDataLmean = a1rChangesDataL.groupby(['Subject', 'LType'])['TempDur'].agg('mean')
a1rFandSDataLmean = a1rFandSDataLmean.reset_index(level=0, drop=True)
dfLmean = pd.DataFrame([[a1rFandSDataLmean['F'], a1rFandSDataLmean['S']]], columns = ['LTypeFMean', 'LTypeSMean'])
aSubjectData = pd.concat([aSubjectData, dfLmean], axis=1)
aSubjectData

,DistanceR,DistanceL,Subject,Label,Gender,LTypeFSum,LTypeSSum,RTypeFSum,RTypeSSum,RTypeFMean,RTypeSMean,LTypeFMean,LTypeSMean
0,13580.30654,13413.296257,111GM3,0,1,15320.0,14520.0,15020.0,14840.0,82.076503,81.092896,89.590643,84.912281


In [33]:
#compute the count of F durations and S durations, grouping by RType
a1rFandSDataRcount = a1rChangesDataR.groupby(['Subject', 'RType'])['TempDur'].agg('count')
a1rFandSDataRcount = a1rFandSDataRcount.reset_index(level=0, drop=True)
dfRcount = pd.DataFrame([[a1rFandSDataRcount['F'], a1rFandSDataRcount['S']]], columns = ['RTypeFcount', 'RTypeScount'])
aSubjectData = pd.concat([aSubjectData, dfRcount], axis=1)
aSubjectData

,DistanceR,DistanceL,Subject,Label,Gender,LTypeFSum,LTypeSSum,RTypeFSum,RTypeSSum,RTypeFMean,RTypeSMean,LTypeFMean,LTypeSMean,RTypeFcount,RTypeScount
0,13580.30654,13413.296257,111GM3,0,1,15320.0,14520.0,15020.0,14840.0,82.076503,81.092896,89.590643,84.912281,183,183


In [34]:
#compute the count of F durations and S durations, grouping by LType
a1rFandSDataLcount = a1rChangesDataL.groupby(['Subject', 'RType'])['TempDur'].agg('count')
a1rFandSDataLcount = a1rFandSDataLcount.reset_index(level=0, drop=True)
dfLcount = pd.DataFrame([[a1rFandSDataLcount['F'], a1rFandSDataLcount['S']]], columns = ['LTypeFcount', 'LTypeScount'])
aSubjectData = pd.concat([aSubjectData, dfLcount], axis=1)
aSubjectData

,DistanceR,DistanceL,Subject,Label,Gender,LTypeFSum,LTypeSSum,RTypeFSum,RTypeSSum,RTypeFMean,RTypeSMean,LTypeFMean,LTypeSMean,RTypeFcount,RTypeScount,LTypeFcount,LTypeScount
0,13580.30654,13413.296257,111GM3,0,1,15320.0,14520.0,15020.0,14840.0,82.076503,81.092896,89.590643,84.912281,183,183,174,168


What kind of cleaning steps did you perform?
The data was already standardised and cleaned. The data was collected from the Karolinska Institute which is Sweden’s forefront academic and medical institution. The dataset consisted of eye movement recordings from 185 children ages 8 and 9 (since recordings for both the left and right eye were extracted from each child, the dataset contained 370 samples). to extract them in to data frame used the L and R value to compute the distance. The distance is computed using sqrt(square(x1-x2) + square(y1-y2). From the data extract few more columns from the eye movement data by adding LType and RType. S-Saccade and F-Fixation: If the distance travelled is < 1 (review this) then it is Fixation else Saccade. Also the eye movements of left and right in the direction of horizontal and vertical. Here three displacements are considered one left, right and no change( L, R, N) and (U, D, N). Also Since the data is recorded per folder we used a loop function to reach out to each folder and collect them all in to one table. That is a data frame with 367 rows x 53 columns

How did you deal with missing values, if any?
There were no missing values since all the features were computed  from the values and so there was no necessary to perform "drop null" or so work on ways to replace values. The data is recorded with every 20 sec and the length of the data depends upon the subject and the reading movement and pattern